# RESDecryptor

This notebook helps in discovering passwords and encryption mechanisms in RESAM. The RESDectyptor module contains some functions that do this but for developing / playing around I find this easier than a (jupyter) console.

The simplest encryption mechanism in RESAM is a substituion cypher (Vigenere) which is used at the task level. Some tasks allow you to set a security context and the UI offers to store this for future use. Different keys are used for different task types but consistency is, as usual, not to be found.

The building block xml seems to have evolved over time, with many little (annoying) differences. Most of teh time the password for the security context is stored in `//tasks/task/settings/password` but this can also be something like `//tasks/task/settings/dbpassword` or `//tasks/task/settings/securitypassword`.

The xpath expression I use for now is  `//*[contains(local-name(), 'password')]` which seems to catch them all (for now).

In [1]:
from resdecryptor import *

somepw = 'abcdefghABCDEFGH1234!@#$'

cyphertexts = [
    '00D000CF00D000C500D300CA00AA00D700AE00AF00A400B200A9008900B600B5009E009300A10098006400AF00900091',
    '00B200AE00B600C700D700CF00D700DC00940093008F009700A800B800B000B800A5008500840080007400A30095008D',
    '00C600D000C700A400B200C700D000D4009400A700B100A80085009300A800B1009D0085009800A20085008000700085',
    '00B400AA00A600D300D200D300C800D600A500B500960097008D008900B600B5009E009300A10098009400930076006C',
    '00C600C400B600C900D700DC00D000CB00A6009900A800A6009800AB00B900BE009A00950098008B008600A200760089'
    ]

In [2]:
for t in cyphertexts:
    print(derive_key(somepw, t))

ommandCommandCommandComm
QLScriptSQLScriptSQLScri
end@MailSend@MailSend@Ma
SHCommandsSSHCommandsSSH
ebServiceWebServiceWebSe


Paramaters can also hold passwords, but they seem to use another algorithm. Since we also have the ability for chosen plain text attacks let's see what we can find out. I created a module with some parameters, extracted the building block and extracted the plain text and corresponding cipher texts.

In [7]:
params = [("a","00C2")
, ("aa","0082009E")
, ("aaa","00FE00CA0085")
, ("b","00C3")
, ("bb","014300EE")
, ("bbb","007B00C60121")
, ("ab","008600A1")
, ("ba","014700F1")
, ("abc","00D800A1013F")
, ("acb","013B20670084")
, ("bac","01180131007E")
, ("bca","015E00AF0088")
, ("cab","013B010000C2")
, ("cba","00DE013F0087")
, ("aaaaaaaaaaaaaaaa","011200BA0159005C01680125006600D30096014E006E00EA006900B800650059")
, ("bbbbbbbbbbbbbbbb","012300A60149006C00E4008100B301142102015B008E010B0066008401010079")
, ("c","00C4")
, ("E","00A6")
, ("F","00A7")
, ("G","00A8")
, ("ac","00BA00A0")
, ("ad","00A600A3")
, ("ae","015200A2")
, ("bc","006F00EF")
, ("bd","014B00F4")
, ("be","007F00F5")
, ("aab","00DB00F60144")
, ("aac","0138006200BF")
, ("aad","00D5008E007E")]


In [10]:
for plain, cipher in sorted(params, key=lambda t: len(t[1])):
    print(plain, [c - p for p, c in zip(string_to_intlist(plain), hexstring_to_intlist(cipher))])

a [97]
b [97]
c [97]
E [97]
F [97]
G [97]
aa [33, 61]
bb [225, 140]
ab [37, 63]
ba [229, 144]
ac [89, 61]
ad [69, 63]
ae [241, 61]
bc [13, 140]
bd [233, 144]
be [29, 144]
aaa [157, 105, 36]
bbb [25, 100, 191]
abc [119, 63, 220]
acb [218, 8196, 34]
bac [182, 208, 27]
bca [252, 76, 39]
cab [216, 159, 96]
cba [123, 221, 38]
aab [122, 149, 226]
aac [215, 1, 92]
aad [116, 45, 26]
aaaaaaaaaaaaaaaa [177, 89, 248, -5, 263, 196, 5, 114, 53, 237, 13, 137, 8, 87, 4, -8]
bbbbbbbbbbbbbbbb [193, 68, 231, 10, 130, 31, 81, 178, 8352, 249, 44, 169, 4, 34, 159, 23]


Looking at strings of length 1 we see a difference of 97 consistently. This could indicate some sort of substitution or something but starting with length 2 we see:
aa [33, 61]
ab [37, 63]
ac [89, 61]
ad [69, 63]
ae [241, 61]
which shows an interesting pattern... let's do more of these.

In [11]:
params2 = [("af","015E00A5")
, ("ag","008A00A4")
, ("ah","007E00A7")
, ("ai","00A200A6")
, ("aj","00A600A9")
, ("ak","014A00A8")
, ("al","014600AB")
, ("bc","006F00EF")
, ("bd","014B00F4")
, ("be","007F00F5")
, ("bf","007300F2")
, ("bg","00A700F3")
, ("bh","008B00F8")
, ("aac","0138006200BF")
, ("aad","00D5008E007E")
, ("aae","0152010200D9")
, ("aaf","006F006600D8")
, ("aag","014C009A01C9")
, ("aah","012900FE0112")
, ("bba","00BE005A00A8")
, ("bbb","007B00C60121")
, ("bbc","013800F2015E")
, ("bbd","0075005E00DF")
, ("bbe","01120092013C")
, ("bbf","01DE00F60095")
, ("bbg","209A006A0132")]

In [12]:
for plain, cipher in sorted(params2):
    print(plain, [c - p for p, c in zip(string_to_intlist(plain), hexstring_to_intlist(cipher))])

aac [215, 1, 92]
aad [116, 45, 26]
aae [241, 161, 116]
aaf [14, 5, 114]
aag [235, 57, 354]
aah [200, 157, 170]
af [253, 63]
ag [41, 61]
ah [29, 63]
ai [65, 61]
aj [69, 63]
ak [233, 61]
al [229, 63]
bba [92, -8, 71]
bbb [25, 100, 191]
bbc [214, 144, 251]
bbd [19, -4, 123]
bbe [176, 48, 215]
bbf [380, 148, 47]
bbg [8248, 8, 203]
bc [13, 140]
bd [233, 144]
be [29, 144]
bf [17, 140]
bg [69, 140]
bh [41, 144]
